# Лабораторная работа №5

## Задание

Провести классификацию найденного датасета, методами решающего дерева и случайного леса.

Импорт библиотек

In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

Загрузка датасета

In [7]:
df = pd.read_csv('cars.csv', encoding='utf-8')
print(df.head(5))

  manufacturer_name model_name transmission   color  odometer_value  \
0            Subaru    Outback    automatic  silver          190000   
1            Subaru    Outback    automatic    blue          290000   
2            Subaru   Forester    automatic     red          402000   
3            Subaru    Impreza   mechanical    blue           10000   
4            Subaru     Legacy    automatic   black          280000   

   year_produced engine_fuel  engine_has_gas engine_type  engine_capacity  \
0           2010    gasoline           False    gasoline              2.5   
1           2002    gasoline           False    gasoline              3.0   
2           2001    gasoline           False    gasoline              2.5   
3           1999    gasoline           False    gasoline              3.0   
4           2001    gasoline           False    gasoline              2.5   

   ... feature_1  feature_2 feature_3 feature_4  feature_5  feature_6  \
0  ...      True       True      True

Удаление ненужных столбцов из датасета

In [8]:
columns_to_drop = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'duration_listed']
df = df.drop(columns=columns_to_drop, axis=1)
print(df.head(5))

  manufacturer_name model_name transmission   color  odometer_value  \
0            Subaru    Outback    automatic  silver          190000   
1            Subaru    Outback    automatic    blue          290000   
2            Subaru   Forester    automatic     red          402000   
3            Subaru    Impreza   mechanical    blue           10000   
4            Subaru     Legacy    automatic   black          280000   

   year_produced engine_fuel  engine_has_gas engine_type  engine_capacity  \
0           2010    gasoline           False    gasoline              2.5   
1           2002    gasoline           False    gasoline              3.0   
2           2001    gasoline           False    gasoline              2.5   
3           1999    gasoline           False    gasoline              3.0   
4           2001    gasoline           False    gasoline              2.5   

   body_type  has_warranty  state drivetrain  price_usd  is_exchangeable  \
0  universal         False  owned 

Удаление строк с пропущенными значениями

In [9]:
df.dropna(inplace=True)

Предобработка данных

In [10]:
# Кодирование категориальных признаков
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])
    
print(df.head(5))

   manufacturer_name  model_name  transmission  color  odometer_value  \
0                 45         763             0      8          190000   
1                 45         763             0      1          290000   
2                 45         519             0      7          402000   
3                 45         609             1      1           10000   
4                 45         664             0      0          280000   

   year_produced  engine_fuel  engine_has_gas  engine_type  engine_capacity  \
0           2010            2           False            1              2.5   
1           2002            2           False            1              3.0   
2           2001            2           False            1              2.5   
3           1999            2           False            1              3.0   
4           2001            2           False            1              2.5   

   body_type  has_warranty  state  drivetrain  price_usd  is_exchangeable  \
0        

In [11]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = df.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
df[numeric_features] = scaler.fit_transform(df[numeric_features])

print(df.head(5))

   manufacturer_name  model_name  transmission     color  odometer_value  \
0           1.089297    0.597314     -1.410285  0.978100       -0.432979   
1           1.089297    0.597314     -1.410285 -0.969685        0.302004   
2           1.089297   -0.149960     -1.410285  0.699845        1.125184   
3           1.089297    0.125674      0.709076 -0.969685       -1.755946   
4           1.089297    0.294117     -1.410285 -1.247940        0.228505   

   year_produced  engine_fuel  engine_has_gas  engine_type  engine_capacity  \
0       0.875318     0.721021           False     0.708498         0.662782   
1      -0.116665     0.721021           False     0.708498         1.407751   
2      -0.240663     0.721021           False     0.708498         0.662782   
3      -0.488659     0.721021           False     0.708498         1.407751   
4      -0.240663     0.721021           False     0.708498         0.662782   

   body_type  has_warranty     state  drivetrain  price_usd  is_exch

Разделение данных на обучающий и тестовый наборы

In [12]:
X = df.drop('feature_0', axis=1)
Y = df['feature_0']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

##### Обучение модели методом решающего дерева

In [13]:
dt_classifier = DecisionTreeClassifier()

Определение сетки параметров для поиска

In [14]:
dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

Поиск оптимальных параметров с использованием кросс-валидации

In [16]:
dt_grid_search = GridSearchCV(dt_classifier, dt_param_grid, cv=5)
dt_grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

Получение лучших параметров

In [17]:
best_dt_params = dt_grid_search.best_params_
print("Лучшие параметры для решающего дерева:", best_dt_params)

Лучшие параметры для решающего дерева: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}


Обучение модели с лучшими параметрами

In [20]:
best_dt_classifier = DecisionTreeClassifier(**best_dt_params)
best_dt_classifier.fit(X_train, Y_train)

DecisionTreeClassifier(max_depth=10, min_samples_split=10)

Предсказание на тестовом наборе данных

In [21]:
Y_pred_dt = best_dt_classifier.predict(X_test)

Оценка модели

In [25]:
print("Отчет о классификации:")
print(classification_report(Y_test, Y_pred_dt))

Отчет о классификации:
              precision    recall  f1-score   support

       False       0.86      0.94      0.90      5958
        True       0.70      0.48      0.57      1747

    accuracy                           0.84      7705
   macro avg       0.78      0.71      0.74      7705
weighted avg       0.83      0.84      0.83      7705



Оценка модели на тестовом наборе данных

In [27]:
dt_accuracy = accuracy_score(Y_test, Y_pred_dt)
print("Точность решающего дерева на тестовом наборе данных:")
print(dt_accuracy)

Точность решающего дерева на тестовом наборе данных:
0.836469824789098


##### Обучение модели методом случайного леса

In [28]:
rf_classifier = RandomForestClassifier()

Определение сетки параметров для поиска

In [29]:
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

Поиск оптимальных параметров с использованием кросс-валидации

In [32]:
rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=5)
rf_grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

Получаение лучших параметров

In [35]:
best_rf_params = rf_grid_search.best_params_
print("Лучшие параметры для случайного леса:", best_rf_params)

Лучшие параметры для случайного леса: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


Обучение модели с лучшими параметрами

In [38]:
best_rf_classifier = RandomForestClassifier(**best_rf_params)
best_rf_classifier.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_split=5,
                       n_estimators=200)

Предсказание на тестовом наборе данных

In [39]:
Y_pred_rf = best_rf_classifier.predict(X_test)

Оценка модели

In [41]:
print("Отчет о классификации:")
print(classification_report(Y_test, Y_pred_rf))

Отчет о классификации:
              precision    recall  f1-score   support

       False       0.88      0.95      0.91      5958
        True       0.74      0.54      0.63      1747

    accuracy                           0.85      7705
   macro avg       0.81      0.74      0.77      7705
weighted avg       0.85      0.85      0.85      7705



Оценка модели на тестовом наборе данных

In [43]:
rf_accuracy = accuracy_score(Y_test, Y_pred_rf)
print("Точность случайного леса на тестовом наборе данных:", rf_accuracy)

Точность случайного леса на тестовом наборе данных: 0.853990914990266
